In [1]:
#Step 1: parse the file to get a seperate file for each gene with its coding sequeces and strain. 
    #get a file for the cds for each gene in the SARS fasta file
#step 2: 
    #a. Break each cds file into subgroups based on strain
    #b. then, for each gene we are going to translate the gene - create a new file where each CDS 
    #has been translated to its amino acid sequence

#so we will have 3 output groups:
    #1. CDS's by each gene type
    #2. Coding Seqs of each subgroup based on the strain
    #3.translated CDS for each gene and for each subgroup

#Step3: Snp mapping - map the snps from the wild type whole genome and get a whole mutated genome file
    #Then we want to do a pairwise alignment for each gene (or just a few) against the mutated genome
    #percent cutoff: we will to generate reports based on different values - 
        #a. 100% = exact matches
        #b. 98-99%
        #c. 95-97%
#look at changes that occur in less than 10 times. 

#Last step: what does it all mean? 

#these are the genes:
"""
    E
    N
    orf1ab
    S
    M
    ORF10
    ORF3a
    ORF8
""" 

IndentationError: unexpected indent (<ipython-input-1-fbac85b49017>, line 24)

In [10]:
#Analysis Pipeline Part 1:
    #Parse the fasta file to pull out each cds for all the genes and strains - save in a file for each gene
    #use the SNP table to map mutations to the reference genome to create a new file with the mutated snps incorporated
    #perform this operation for each genome strain
    #Parse the coding sequences for ORF1ab into its corresponding polypeptide seqeunce - call it ORF1ab_PP.fasta

from Bio import SeqIO
import os
import re
   
def parse_fasta_per_gene (input_file):
    with open (input_file, 'r') as input_file:
        output_dir = os.getcwd()
        if not os.path.isdir(output_dir + '/SARS-CoV2_CDS_Outputfile/' ):
            os.mkdir(output_dir + '/SARS-CoV2_CDS_Outputfile/')

        Genes = ['E','M','N', 'NS3','NS6','NS7a','NS7b','NS8','orf1a','orf1ab','S','ORF10','ORF3a','ORF6','ORF7','ORF7a','ORF8']
        list = []
        #loop through each element to split
        for element in input_file:
            list.append(element)           
        line = ''.join(list)
        x = line.split('>')
        for i in Genes:
            symbol = 'Gene Symbol:'+ i
            filename = symbol + '-' + '.fasta'
            filename = filename.replace('Gene Symbol:', 'Gene_and_Strains_')
            #open outfile for writing
            out_file = open(output_dir + '\\SARS-CoV2_CDS_Outputfile\\' + filename, 'w')
            for line in x:
                if symbol in line:
                    out_file.write(str('>') + line)
            out_file.close()
        print("Files Saved in Directory")

In [11]:
#call the function for Gene
parse_fasta_per_gene("SARS-CoV-2-CdsFastaResults.fasta")

Files Saved in Directory


In [4]:
from Bio import SeqIO
import os
import re
def parse_fasta_strain (input_file):
    with open (input_file, 'r') as input_file:
        #open outfile for writing
        output_dir = os.getcwd()
        if not os.path.isdir(output_dir + '/SARS-CoV2_Orf1ab_CDS_Strains/' ):
            os.mkdir(output_dir + '/SARS-CoV2_Orf1ab_CDS_Strains/')
        list = []
        #loop through each element to split
        for element in input_file:
            list.append(element)           
        line = ''.join(list)
        x = line.split('gb')
        for line in x:
                if 'Symbol:orf1ab' in line:
                    strain = str(re.findall(r'Strain Name:\w+', str(line)))
                    filename = strain+"_"+".fasta"
                    filename = filename.replace('/', '_')
                    filename = filename.replace('[', '')
                    filename = filename.replace(']', '')
                    filename = filename.replace("'", '')
                    filename = filename.replace('/', '')
                    filename = filename.replace('Strain Name:', 'Orf1ab_Strain_')
                    out = open(output_dir + '\\SARS-CoV2_Orf1ab_CDS_Strains\\' + filename, 'w')
                    out.write(str('>gb')+line)
                    out.close()
        print("Files Saved in Output directory")

In [5]:
parse_fasta_strain('Gene_and_Strains_orf1ab-.fasta')

Files Saved in Output directory


In [ ]:
#repeat the above code to get the strains for all of ORF1a as well

In [6]:
#Get full genome sequence from file and turn into a list for parsing
import os
def Map_Mutations (input_file):
    with open (input_file, 'r') as input_file:
        #open fasta file to get the genome for reading
        #remove the first line
        header = input_file.readline()
        #turn the sequence into a list
        wild_type_list = [x for line in input_file.read().split('\n') for x in line] 
        #print the first mutation location to check that it is the wildtype file
        print(wild_type_list[240])
   
    #Then open the SNP location file for reading to get the mutated genome from the locations
    with open ('countResult.txt', 'r') as input_file2:
        #remove the first line
        input_file2.readline()
        #open a file to write the final mutated genome
        out = open('NC_045512_mutated.txt', 'w') 
        #loop through the snp location file and split at the :
        #then get insert the mutated snp at the index location in the wild type genome list 
        for i in input_file2.readlines():
            coord, mutate = i.split(':')
            wild_type_list[int(coord)-1] = mutate[4]
        print(wild_type_list[240])
        mutation = "".join(wild_type_list)

        output_dir = os.getcwd()
        if not os.path.isdir(output_dir + '/Mutated_fasta/' ):
            os.mkdir(output_dir + '/Mutated_fasta/')
        with open(output_dir + '\\Mutated_fasta\\' + 'NC_045512_Mutated.fasta', "w") as f:
            f.write(str('>2019-nCoV|WH01|NC_045512|2020-01-05') + '\n'+ mutation)
        print( '\n'+ "Files Saved in Directory" + '\n')
        
        

In [7]:
Map_Mutations('NC_045512.fasta')

C
T

Files Saved in Directory



In [ ]:
def Mutated_CDS_per_Genes(input_file):
    with open (input_file, 'r') as f:   #Just make sure to change this file to the mutated file 
        data = f.read()
        #get the mutated cds for each gene
        #slide at the locations of each gene from the genbank file
        ORFlab = (data[265:21555])
        #write the sliced data to the outfile as the mutated cds for each gene
        out.write('Mutated CDS for Gene ORF1ab' + '\n' + str(ORFlab) + '\n')
        S = (data[21562:25384])
        out.write('Mutated CDS for Gene S' + '\n' + str(S) + '\n')
        ORF3a = (data[25392:26220])
        out.write('Mutated CDS for Gene ORF3a' + '\n' + str(ORF3a) + '\n')
        E = (data[26244:26472])
        out.write('Mutated CDS for Gene E' + '\n' + str(E) + '\n')
        M = (data[26522:27191])
        out.write('Mutated CDS for Gene M' + '\n' + str(M) + '\n')
        ORF6 = (data[27201:27387])
        out.write('Mutated CDS for Gene ORF6' + '\n' + str(ORF6) + '\n')
        ORF7a = (data[27393:27759])
        out.write('Mutated CDS for Gene ORF7a' + '\n' + str(ORF7a) + '\n')
        ORF7b = (data[27755:27887])
        out.write('Mutated CDS for Gene ORF7b' + '\n' + str(ORF7b) + '\n')
        ORF8 = (data[27893:28259])
        out.write('Mutated CDS for Gene ORF8' + '\n' + str(ORF8) + '\n')
        N = (data[28273:29533])
        out.write('Mutated CDS for Gene N' + '\n' + str(N) + '\n')
        ORF10 = (data[29557:29674])
        
        out.write('Mutated CDS for Gene ORF10' + '\n' + str(ORF10) + '\n')
        out.close()
             

In [5]:
#Translate each CDS to get the protein sequences 
from Bio import SeqIO
def parse_fasta_protein (input_file):
    input_file = SeqIO.parse(input_file, 'fasta')
    #with open(out_file, 'w') as out_file:
    # DNA codon table
    protein = {"TTT" : "F", "CTT" : "L", "ATT" : "I", "GTT" : "V",
           "TTC" : "F", "CTC" : "L", "ATC" : "I", "GTC" : "V",
           "TTA" : "L", "CTA" : "L", "ATA" : "I", "GTA" : "V",
           "TTG" : "L", "CTG" : "L", "ATG" : "M", "GTG" : "V",
           "TCT" : "S", "CCT" : "P", "ACT" : "T", "GCT" : "A",
           "TCC" : "S", "CCC" : "P", "ACC" : "T", "GCC" : "A",
           "TCA" : "S", "CCA" : "P", "ACA" : "T", "GCA" : "A",
           "TCG" : "S", "CCG" : "P", "ACG" : "T", "GCG" : "A",
           "TAT" : "Y", "CAT" : "H", "AAT" : "N", "GAT" : "D",
           "TAC" : "Y", "CAC" : "H", "AAC" : "N", "GAC" : "D",
           "TAA" : "STOP", "CAA" : "Q", "AAA" : "K", "GAA" : "E",
           "TAG" : "STOP", "CAG" : "Q", "AAG" : "K", "GAG" : "E",
           "TGT" : "C", "CGT" : "R", "AGT" : "S", "GGT" : "G",
           "TGC" : "C", "CGC" : "R", "AGC" : "S", "GGC" : "G",
           "TGA" : "STOP", "CGA" : "R", "AGA" : "R", "GGA" : "G",
           "TGG" : "W", "CGG" : "R", "AGG" : "R", "GGG" : "G" 
           }
    protein_sequence = ""
    for line in input_file:
        if symbol in line.description:  
            Seq1 = line.seq
    for i in range(0, len(Seq1)-(3+len(Seq1)%3), 3):
        if protein[Seq1[i:i+3]] == "STOP" :
            break
        protein_sequence += protein[Seq1[i:i+3]]
        print(protein_sequence)



In [6]:
parse_fasta_protein("SARS-CoV-2-CdsFastaResults.fasta")

NameError: name 'symbol' is not defined